<a href="https://colab.research.google.com/github/ibonetc/AI_Deep-Learning/blob/main/Transformers_HuggingFace_Imagenes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformers

- Clasificación de imágenes
- Traducir audios
- De texto a imágenes

**Nota**: Intalar estas bibliotecas y reiniciar el entorno

In [ ]:
!pip install transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.9 MB/s eta 0:00:00


In [ ]:
!pip install transformers
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.9 MB/s eta 0:00:00


In [ ]:
import transformers
print(transformers.__version__)

4.33.1


In [ ]:
from transformers import pipeline

## Clasificación de imágenes

In [ ]:
vision_classifier = pipeline(task="image-classification", model='google/vit-base-patch16-224')
prediction=vision_classifier(images="/content/drive/MyDrive/Fotos_Pruebas/cisneNegro.jpg")
print('Clase:', prediction[0]['label'] , prediction[0]['score']*100)

Clase: black swan, Cygnus atratus 99.56654906272888


In [ ]:
prediction

[{'score': 0.9956654906272888, 'label': 'black swan, Cygnus atratus'},
 {'score': 0.0003855983086396009, 'label': 'goose'},
 {'score': 0.00024381582625210285, 'label': 'lakeside, lakeshore'},
 {'score': 0.00020543277787510306,
  'label': 'American coot, marsh hen, mud hen, water hen, Fulica americana'},
 {'score': 9.251195297110826e-05,
  'label': 'European gallinule, Porphyrio porphyrio'}]

In [ ]:
from transformers import ViTImageProcessor, ViTForImageClassification
from PIL import Image

In [ ]:
image = Image.open("/content/drive/MyDrive/Fotos_Pruebas/cisneNegro.jpg")

In [ ]:
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

inputs = processor(images=image, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
# model predicts one of the 1000 ImageNet classes
predicted_class_idx = logits.argmax(-1).item()
print("Clase:", model.config.id2label[predicted_class_idx])

Clase: black swan, Cygnus atratus


## Traducir audios

In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from datasets import Audio, load_dataset
from IPython import display

In [ ]:
# Cargar el modelo y procesador
processor = WhisperProcessor.from_pretrained("openai/whisper-large-v2")
model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-large-v2")
forced_decoder_ids = processor.get_decoder_prompt_ids(language="spanish", task="transcribe")

In [ ]:
## Cargar los datos
#ds = load_dataset("common_voice", "es", split="test", streaming=True)
dataset = load_dataset("mozilla-foundation/common_voice_11_0", "es", split="test", streaming=True )

In [ ]:
sampling_rate=16_000
ds = dataset.cast_column("audio", Audio(sampling_rate=sampling_rate))

In [ ]:
## Cargar primer audio
input_speech = next(iter(ds))["audio"]
input_features = processor(input_speech["array"], sampling_rate=input_speech["sampling_rate"], return_tensors="pt").input_features

Reading metadata...: 15520it [00:00, 19360.91it/s]


In [ ]:
# generar ids de tokens
predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
# decodificar a texto
transcription1 = processor.batch_decode(predicted_ids)

transcription2 = processor.batch_decode(predicted_ids, skip_special_tokens=True)

In [ ]:
print(transcription1)
print(transcription2)

['<|startoftranscript|><|es|><|transcribe|><|notimestamps|> Habitan aguas poco profundas y rocosas.<|endoftext|>']
[' Habitan aguas poco profundas y rocosas.']


In [ ]:
input_speech

{'path': 'es_test_0/common_voice_es_19698530.mp3',
 'array': array([ 5.09317033e-11,  1.16415322e-10,  1.01863407e-10, ...,
        -5.49094111e-05,  2.18403147e-04,  7.60999246e-05]),
 'sampling_rate': 16000}

In [ ]:
display.display(display.Audio(input_speech["array"], rate=sampling_rate, autoplay=True))

In [ ]:
audios=[]
i=0
N=4  ## Hacer los primeros 4
for audio in ds:
  input_features = processor(audio["audio"]["array"], sampling_rate=audio["audio"]["sampling_rate"], return_tensors="pt").input_features
  audios.append(audio["audio"]["array"])
  predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
  transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
  print(transcription)
  i+=1
  if (i==N):
    break


Reading metadata...: 15520it [00:00, 62696.70it/s]


[' Habitan aguas poco profundas y rocosas.']
[' Opera principalmente vuelos de cargotajes y regionales de carga.']
[' Tres.']
[' Realicé los estudios primarios en Francia para continuar luego en España.']


In [ ]:
for a in audios:
  display.display(display.Audio(a, rate=sampling_rate, autoplay=True))

### Un audio

In [ ]:
audio=display.Audio("/content/te-gusta-el-azul.mp3", rate=16000, autoplay=True)
display.display(audio)

In [ ]:
!pip install pydub

In [ ]:
from pydub import AudioSegment
import numpy as np

In [ ]:
# Cargar audio
audio = AudioSegment.from_mp3('/content/te-gusta-el-azul.mp3')

# Resample a 1600 Hz
audio = audio.set_frame_rate(16000)

# Convertir a numpy array
numpy_audio = np.array(audio.get_array_of_samples())

In [ ]:
input_features = processor(numpy_audio, sampling_rate=16000, return_tensors="pt").input_features

In [ ]:
predicted_ids = model.generate(input_features, forced_decoder_ids=forced_decoder_ids)
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
print(transcription)

[' ¿Te gusta el azul?']
